In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, QWEN2_VL, VLM_IMAGE_TOKENS, \
    Qwen2_VL_process_fn, LLAVA_QWEN2, FastVLM_process_fn

from PIL import Image
import numpy as np
import torch
import math
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

from transformers.image_transforms import (
    convert_to_rgb,
    resize,
)

/workspace/VLM_Embed/vlm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/VLM_Embed/vlm/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM_Embed/src/model/vlm_backbone/internvideo2/modeling_internvideo2.py:541: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
[2026-01-01 12:07:48,211] DEBUG [matplotlib:342] matplotlib data path: /workspace/VLM_Embed/vlm/lib/python3.12/site-packages/matplotlib/mpl-data
[2026-01-01 12:07:48,217] DEBUG [matplotlib:342] CONFIG

FusedMLP of flash_attn is not installed!!!
DropoutAddRMSNorm of flash_attn is not installed!!!
flash_attn_interface or bert_padding of flash_attn is not installed!!!


[2026-01-01 12:07:48,424] DEBUG [matplotlib.pyplot:496] Loaded backend module://matplotlib_inline.backend_inline version unknown.
[2026-01-01 12:07:48,427] DEBUG [matplotlib.pyplot:496] Loaded backend inline version unknown.


In [10]:
# model_args = ModelArguments(
#     model_name='Qwen/Qwen2-VL-2B',
#     checkpoint_path='TIGER-Lab/VLM2Vec-Qwen2VL-2B',
#     pooling='last',
#     normalize=True,
#     model_backbone='qwen2_vl',
#     lora=True
# )
model_args = ModelArguments(
    model_name='apple/FastVLM-0.5B',
    pooling='last',
    normalize=True,
    model_backbone=LLAVA_QWEN2,
)
data_args = DataArguments()

processor = load_processor(model_args, None)
model = MMEBModel.load(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()

[2026-01-01 12:12:33,194] INFO [src.utils:21] Loading processor from: apple/FastVLM-0.5B


Processor load here for LLAVA-QWEN2


[2026-01-01 12:12:33,487] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
[2026-01-01 12:12:33,508] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/tokenizer_config.json HTTP/1.1" 200 0
[2026-01-01 12:12:33,788] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/models/apple/FastVLM-0.5B/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64
[2026-01-01 12:12:34,489] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/config.json HTTP/1.1" 307 0
[2026-01-01 12:12:34,506] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/config.json HTTP/1.1" 200 0
[2026-01-01 12:12:34,776] DEBUG [urllib3.connect

Detected model type: llava_qwen2
Determined model backbone: llava_qwen2


[2026-01-01 12:12:35,396] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/config.json HTTP/1.1" 307 0
[2026-01-01 12:12:35,412] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/config.json HTTP/1.1" 200 0
[2026-01-01 12:12:35,686] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/llava_qwen.py HTTP/1.1" 307 0
[2026-01-01 12:12:35,704] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/llava_qwen.py HTTP/1.1" 200 0
[2026-01-01 12:12:36,007] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/config.json HTTP/1.1" 307 0
[2026-01-01 12:12:36,025] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/mo

Loaded base model from HF: apple/FastVLM-0.5B


MMEBModel(
  (encoder): LlavaQwen2ForCausalLM(
    (model): LlavaQwen2Model(
      (embed_tokens): Embedding(151936, 896)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
   

In [16]:
processor.image_token

'<image>'

In [4]:
model.config

Qwen2VLConfig {
  "architectures": [
    "Qwen2VLForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "image_token_id": 151655,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2_vl",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": {
    "mrope_section": [
      16,
      24,
      24
    ],
    "rope_type": "default",
    "type": "default"
  },
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "text_config": {
    "architectures": [
      "Qwen2VLForConditionalGeneration"
    ],
    "attention_dropout": 0.0,
    "bos_token_id": 151643,
    "dtype": "bfloat16",
    "eos_token_id": 151645,
    "hidden_act": "silu",
    "hidden_size": 1536,
    "image_token_id": null,
    "initializer_

In [14]:
vision_tower = model.encoder.get_vision_tower()
vision_tower.config['image_cfg']['patch_size']

64

In [ ]:
x = torch.randn(1, 3, 768, 768).to('cuda', dtype=torch.bfloat16)
with torch.no_grad():
    y = model.encoder.get_vision_tower()(x)
y.shape

RuntimeError: Given input size: (3072x12x12). Calculated output size: (3072x0x0). Output size is too small